In [2]:

import pandas as pd
import numpy as np

df = pd.read_csv('podcastdata_dataset.csv')
def preprocess_tolower(text):
    text=text.lower()
    return text

df['text']=df['text'].apply(preprocess_tolower)

In [3]:
from llama_index.core.node_parser import (
    SentenceSplitter,
)
from llama_index.core import Document 
from llama_index.core.node_parser import LangchainNodeParser
from langchain.text_splitter import RecursiveCharacterTextSplitter 
from langchain.embeddings import HuggingFaceBgeEmbeddings

In [4]:
docs = [
    Document(
        text=row['text'],
        metadata={
            'Title': row['title'],
            'Guest': row['guest'],
            'id': row['id']
        }
    )
    for _, row in df[:3].iterrows()
]

parser = LangchainNodeParser(RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap = 200
    )
)
nodes = parser.get_nodes_from_documents(docs)

In [11]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
test_emeds = embed_model.get_text_embedding("Hello World!")

/Users/hritik/lex_llm/rag_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [13]:
test_emeds

[-0.003275706898421049,
 -0.011690810322761536,
 0.04155922308564186,
 -0.03814813122153282,
 0.02418307028710842,
 0.013644246384501457,
 0.011117849498987198,
 0.04811973497271538,
 0.021409526467323303,
 0.014174937270581722,
 -0.003071074141189456,
 -0.03843548148870468,
 0.009438999928534031,
 0.04921751841902733,
 0.03807622939348221,
 0.022404132410883904,
 0.03524315357208252,
 -0.013048947788774967,
 -0.10893471539020538,
 -0.02066672220826149,
 0.10619361698627472,
 0.047313589602708817,
 -0.02044331468641758,
 -0.04986639693379402,
 0.001053651561960578,
 -0.015180852264165878,
 -0.014586982317268848,
 0.035476699471473694,
 -0.0015565998619422317,
 -0.17595292627811432,
 -0.013892754912376404,
 -0.011971266940236092,
 0.0968281552195549,
 -0.010525037534534931,
 0.03767592832446098,
 -0.018267668783664703,
 -0.0025102065410465,
 0.04775373637676239,
 -0.044650230556726456,
 0.008240804076194763,
 0.041858285665512085,
 -0.01266651414334774,
 -0.001488077687099576,
 -0.00966

In [12]:
from llama_index.core import ServiceContext, set_global_service_context
ctx_base = ServiceContext.from_defaults(
    llm=None,
    embed_model=embed_model
)

LLM is explicitly disabled. Using MockLLM.


/var/folders/6y/tpltm_9x7xs0rm16k52xxqbw0000gn/T/ipykernel_80834/2198921486.py:2: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  ctx_base = ServiceContext.from_defaults(


In [14]:

from llama_index.core import VectorStoreIndex
sentence_index = VectorStoreIndex(nodes, service_context=ctx_base)

In [16]:

for node in nodes:
    text = node.get_text()  
    embedding = embed_model.get_text_embedding_batch([text])
    print(f"Text: {text}\nEmbedding: {embedding}\n")

Text: as part of mit course 6s099, artificial general intelligence, i've gotten the chance to sit down with max tegmark. he is a professor here at mit. he's a physicist, spent a large part of his career studying the mysteries of our cosmological universe. but he's also studied and delved into the beneficial possibilities and the existential risks of artificial intelligence. amongst many other things, he is the cofounder of the future of life institute, author of two books, both of which i highly recommend. first, our mathematical universe. second is life 3.0. he's truly an out of the box thinker and a fun personality, so i really enjoy talking to him. if you'd like to see more of these videos in the future, please subscribe and also click the little bell icon to make sure you don't miss any videos. also, twitter, linkedin, agi.mit.edu if you wanna watch other lectures or conversations like this one. better yet, go read max's book, life 3.0. chapter seven on goals is my favorite. it's r